# Melting and dissolution at a stagnation point

This notebook presents code to simulate melting and dissolution at a stagnation point using Dedalus.
We present a moving boundary problem and phase-field model formulation, and compare.

# Imports

In [ ]:
import numpy as np
import dedalus.public as de
import matplotlib.pyplot as plt
import interpolation as ip
import field_tools as flt
import file_tools as flts

import logging
root = logging.root
for h in root.handlers: h.setLevel("WARNING")
logger = logging.getLogger(__name__)

d = de.operators.differentiate
interp = de.operators.interpolate
integrate = de.operators.integrate

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm
cmap = cm.get_cmap('Greys')
color = cmap(.5)

cdict_phi_grey = { 
    'red':  ((0.0, 1., 1.), (0.5, .7,.7), (1.0, color[0], color[0])),
    'green':((0.0, 1., 1.), (0.5, .7,.7), (1.0, color[0], color[0])),
    'blue': ((0.0, 1., 1.), (0.5, .7,.7), (1.0, color[0], color[0])),}
#     'alpha':((0.0, 1., 0.0), (0.1, 0.0, 0.0), (1.0, 1., 1.))  }
cmap_phi = LinearSegmentedColormap('cmap_phi', cdict_phi_grey)
plt.register_cmap(cmap=cmap_phi)

# Sharp simulation

In [ ]:
def melting_salty_stag(ν,κ,μ,S,M,D,nx):
    xbasis = de.Chebyshev('x',nx,interval=(0,1),dealias=3/2)
    domain = de.Domain([xbasis],grid_dtype=np.float64)
    x, = domain.grids(domain.dealias)

    problem = de.NLBVP(domain,variables=['u','ux','uxx','v','Tl','Tlx','Ts','Tsx','C','Cx'])
    problem.meta[:]['x']['dirichlet'] = True
    problem.parameters['ν'] = ν
    problem.parameters['κ'] = κ
    problem.parameters['μ'] = μ
    problem.parameters['M'] = M
    problem.parameters['S'] = S
    problem.parameters['D'] = D

    problem.add_equation("dx(v) = 0")
    problem.add_equation('ux - dx(u) = 0')
    problem.add_equation('uxx- dx(ux) = 0')
    problem.add_equation("Tlx- dx(Tl) = 0")
    problem.add_equation("Tsx- dx(Ts) = 0")
    problem.add_equation("Cx - dx(C) = 0")
    problem.add_equation('ν*dx(uxx) = 1 + uxx*(u-v) - ux*ux')
    problem.add_equation("κ*dx(Tlx) = (u-v)*Tlx")
    problem.add_equation("κ*dx(Tsx) = -v*Tsx")
    problem.add_equation("μ*dx(Cx) = (u-v)*Cx")

    problem.add_bc('right(ux) = -1')
    problem.add_bc("right(Tl) = 1")
    problem.add_bc("right(C) = 1")
    problem.add_bc('left(u) = 0')
    problem.add_bc('left(ux) = 0')
    problem.add_bc("left(Tl + M*C) = 0")
    problem.add_bc("left(Tl) - right(Ts) = 0")
    problem.add_bc("κ*(left(Tlx) - right(Tsx)) + right(v)*S = 0")
    problem.add_bc("μ*left(Cx) = -left(v*C)")
    problem.add_bc("left(Ts) = -D")

    solver = problem.build_solver()
    u, ux, uxx, v, Tl, Tlx, Ts, Tsx, C, Cx = [solver.state[name] for name in problem.variables]
    for field in [u, ux, uxx, v, Tl, Tlx, Ts, Tsx, C, Cx]: field.set_scales(domain.dealias)

    u['g'], ux['g'], uxx['g'] = -x, -1, 0
    Tl['g'] = 1
    Ts['g'] = -D
    C['g'] = 1

    tolerance = 1e-10
    pert = solver.perturbations.data
    pert.fill(1+tolerance)

    while np.sum(np.abs(pert)) > tolerance:
        print(np.sum(np.abs(pert)))
        solver.newton_iteration()
    
    sim = {'x':x,'domain':domain,'ul':u,'ulx':ux,'ulxx':uxx,'v':v,'Tl':Tl,'Tlx':Tlx,'Ts':Ts,'Tsx':Tsx,'Cl':C,'Clx':Cx}
    params = {'S':S,'ν':ν,'κ':κ,'μ':μ,'M':M,'D':D}
    return sim, params

## Example plot

In [ ]:
sim0, params0 = melting_salty_stag(1e-2,1e-3,1e-3,1,1,1,64)

plt.plot(sim0['x'],sim0['Tl']['g'],label='$T^+$')
plt.plot(sim0['x']-1,sim0['Ts']['g'],label='$T^-$')
plt.plot(sim0['x'],sim0['Cl']['g'],label='$C$')
plt.plot(sim0['x'],sim0['ul']['g'],label='$u$')
plt.legend()


# Phase-field model

In [ ]:
def melting_salty_stag_phase(ν,κ,μ,S,M,D,ϵ,α,γ,η,n,δ=1e-3,thresh=500,guesses=None,damping=.1):
    xbasis = de.Chebyshev('x',n,interval=(0,1),dealias=2)
    domain = de.Domain([xbasis],grid_dtype=np.float64)
    x, = domain.grids(domain.dealias)

    problem = de.NLBVP(domain,variables=['ul','ulx','ulxx',
                                         'us','usx','usxx',
                                         'Tl','Tlx','Ts','Tsx',
                                         'Cl','Clx','Cs','Csx',
                                         'fl','flx','fs','fsx',
                                         'v'])
    problem.meta[:]['x']['dirichlet'] = True
    problem.parameters['ν'] = ν
    problem.parameters['κ'] = κ
    problem.parameters['μ'] = μ
    problem.parameters['S'] = S
    problem.parameters['M'] = M
    problem.parameters['D'] = D
    problem.parameters['eps'] = ϵ
    problem.parameters['α'] = α
    # problem.parameters['β'] = β
    problem.parameters['η'] = η
    problem.parameters['δ'] = δ
    problem.parameters['γ'] = γ

    problem.add_equation('κ*dx(Tlx) = ((1-fl)*ul - v)*Tlx + S*v*flx')
    problem.add_equation('κ*dx(Tsx) = ((1-fs)*us - v)*Tsx + S*v*fsx')
    problem.add_equation('μ*dx(Clx) = (ul - v)*Clx - dx(log(1-fl+δ))*(μ*Clx + v*Cl)')
    problem.add_equation('μ*dx(Csx) = (us - v)*Csx - dx(log(1-fs+δ))*(μ*Csx + v*Cs)')
    problem.add_equation('γ*dx(flx) = -α*v*flx + (γ/eps**2)*fl*(1-fl)*(1-2*fl) + fl*(1-fl)*(Tl+M*Cl)/eps')# 
    problem.add_equation('γ*dx(fsx) = -α*v*fsx + (γ/eps**2)*fs*(1-fs)*(1-2*fs) + fs*(1-fs)*(Ts+M*Cs)/eps')#
    problem.add_equation('ν*dx(ulxx) = 1 + ulxx*(ul-v) - ulx*ulx + fl*ulx/η')
    problem.add_equation('ν*dx(usxx) = 1 + usxx*(us-v) - usx*usx + fs*usx/η')
    problem.add_equation('Tlx - dx(Tl) = 0')
    problem.add_equation('Tsx - dx(Ts) = 0')
    problem.add_equation('Clx - dx(Cl) = 0')
    problem.add_equation('Csx - dx(Cs) = 0')
    problem.add_equation('flx - dx(fl) = 0')
    problem.add_equation('fsx - dx(fs) = 0')
    problem.add_equation('ulx - dx(ul) = 0')
    problem.add_equation('ulxx- dx(ulx)= 0')
    problem.add_equation('usx - dx(us) = 0')
    problem.add_equation('usxx- dx(usx)= 0')
    problem.add_equation('dx(v) = 0')

    problem.add_bc('right(Tl) = 1')
    problem.add_bc('right(Cl) = 1')
    problem.add_bc('right(fl) = 0')
    problem.add_bc('right(ulx) = -1')
    problem.add_bc('left(Tl) - right(Ts) = 0')
    problem.add_bc('left(Tlx)- right(Tsx) = 0')
    problem.add_bc('left(Cl) - right(Cs) = 0')
    problem.add_bc('left(Clx)- right(Csx) = 0')
    problem.add_bc('left(fl) - right(fs) = 0')
    problem.add_bc('left(flx)- right(fsx) = 0')
    problem.add_bc('left(ul) - right(us) = 0')
    problem.add_bc('left(ulx)- right(usx) = 0')
    problem.add_bc('left(ulxx)-right(usxx) = 0')
    problem.add_bc('left(fl) = 1/2')
    problem.add_bc('left(Ts) = -D')
    problem.add_bc('left(Cs) = 0')
    problem.add_bc('left(fs) = 1')
    problem.add_bc('left(us) = 0')
    problem.add_bc('left(usx) = 0')

    solver = problem.build_solver()

    fields = {}
    for a in problem.variables:
        fields[a] = solver.state[a]
        fields[a].set_scales(domain.dealias)
        fields[a]['g'] = 0
        
    fl,fs = fields['fl'], fields['fs']
    fl['g'] = (1/2)*(1-np.tanh(x/(2*ϵ)))
    fs['g'] = (1/2)*(1-np.tanh((x-1)/(2*ϵ)))    
    if guesses:
        for a in fields:
            fields[a]['g'] = ip.interp(guesses[a],x)
    else:
        sim0, params0 = melting_salty_stag(ν,κ,μ,S,M,D,128)
        ul0,Tl0,Ts0,Cl0,v0 = [sim0[f] for f in ['ul','Tl','Ts','Cl','v']]
        fields['ul']['g'] = ip.interp(ul0,x)
        fields['us']['g'] = 0
        fields['Tl']['g'] = ip.interp(Tl0,x)
        fields['Ts']['g'] = ip.interp(Ts0,x)
        fields['Cl']['g'] = ip.interp(Cl0,x)
        fields['Cs']['g'] = Cl0.interpolate(x='left')['g'][0]
        fields['v']['g'] = v0['g'][0]        
    for f in ['ul','us','ulx','usx','Tl','Ts','Cl','Cs']:
        fields[f+'x']['g'] = fields[f].differentiate('x')['g']

    tolerance = 1e-10
    pert = solver.perturbations.data
#     pert.fill(1+tolerance)
    res = 1#np.sum(np.abs(pert))

    while res > tolerance:
        solver.newton_iteration(damping=damping)
        res = np.sum(np.abs(pert))
        if res < thresh: damping = 1
        print(res)
    
    sim = {'x':x,'domain':domain}
    for field in fields: sim[field] = fields[field]
    params = {'S':S,'ν':ν,'κ':κ,'μ':μ,'M':M,'D':D,
              'ϵ':ϵ,'α':α,'γ':γ,'η':η}
    return sim, params

## Example plot

In [ ]:
ν,κ,μ = 1e-1,1e-1,1e-1
S,N,D = 1,1,1
ϵ = 0.05
α = ϵ*(S/κ)*5/6
γ = 1
β = 4/2.648228
η = (β*ϵ)**2/ν
n = 64
δ = 2e-5

sim0, params0 = melting_salty_stag(ν,κ,μ,S,N,D,128)
sim1, params1 = melting_salty_stag_phase(ν,κ,μ,S,N,D,ϵ,α,γ,η,n,δ=δ,thresh=100)

In [ ]:
i = 1
x0, Tl0, Ts0, Cl0, ul0 = [sim0[name] for name in ['x','Tl','Ts','Cl','ul']]
x, Tl, Ts, Cl, Cs, ul, us, fl, fs = [sim1[name] for name in ['x','Tl','Ts','Cl','Cs','ul','us','fl','fs']]
Lx = -1
fig, ax = plt.subplots(2,2,figsize=(7,3.5),sharex=True)
ax[0,0].plot(x,Tl['g'],'C1--',label=f'$\\varepsilon = {ϵ:.2f}$')
ax[0,0].plot(x+Lx,Ts['g'],'C1--')
ax[0,0].plot(x0,Tl0['g'],'C1',label='Exact')
ax[0,0].plot(x0+Lx,Ts0['g'],'C1')
ax[0,0].fill_between(x,-1,2*fl['g']-1,color='lightgray')
ax[0,0].fill_between(x+Lx,-1,2*fs['g']-1,color='lightgray')
ax[0,0].set(xlim=[-1,1],ylim=[-1,1],xlabel='$x$',ylabel='$T$',title='Temperature $T$')
ax[0,0].legend(frameon=False)

ax[0,1].plot(x,Cl['g'],'C2--',label=f'$\\varepsilon = {ϵ:.2f}$')
ax[0,1].plot(x+Lx,Cs['g'],'C2--')
ax[0,1].plot(x0,Cl0['g'],'C2',label='Exact')
ax[0,1].fill_between(x,0,fl['g'],color='lightgray')
ax[0,1].fill_between(x+Lx,0,fs['g'],color='lightgray')
ax[0,1].set(xlim=[-1,1],ylim=[0,1],xlabel='$x$',ylabel='$C$',title='Salinity $C$')
ax[0,1].legend(frameon=False)

ax[1,0].plot(x,-ul['g'],'C0--',label=f'$\\varepsilon = {ϵ:.2f}$')
ax[1,0].plot(x+Lx,-us['g'],'C0--')
ax[1,0].plot(x0,-ul0['g'],'C0',label='Exact')
ax[1,0].fill_between(x,0,fl['g'],color='lightgray')
ax[1,0].fill_between(x+Lx,0,fs['g'],color='lightgray')
ax[1,0].set(xlim=[-1,1],ylim=[0,1],xlabel='$x$',ylabel='$u$',title='Normal velocity $u$')
ax[1,0].legend(frameon=False)

fl0 = 0.5*(1-np.tanh((x0+Lx)/(2*ϵ)))
fs0 = 0.5*(1-np.tanh((x0)/(2*ϵ)))
ax[1,1].plot(x,fl['g'],'k--',label=f'$\\varepsilon = {ϵ:.2f}$')
ax[1,1].plot(x+Lx,fs['g'],'k--')
ax[1,1].plot(x0+Lx,fl0,'k',label='Exact')
ax[1,1].plot(x0,fs0,'k')
ax[1,1].fill_between(x,0,fl['g'],color='lightgray')
ax[1,1].fill_between(x+Lx,0,fs['g'],color='lightgray')
ax[1,1].set(xlim=[-1,1],ylim=[0,1],xlabel='$x$',ylabel=r'$\phi$',title='Phase field $\\phi$')
ax[1,1].legend(loc='upper left',frameon=False)

ax[1,1].text(-.4,.3,f'$v_0 = {-sim0["v"]["g"][0]:.3f}$',horizontalalignment='right')
ax[1,1].text(-.4,.15,f'$v = {-sim1["v"]["g"][0]:.3f}$',horizontalalignment='right')

plt.tight_layout()
plt.savefig('melt-salt-flow-diagram.pdf',bbox_inches='tight')